## 1. Configuration initiale

In [0]:
import os
from pyspark.sql.functions import year, month, to_date, col

print("="*60)
print("🔧 CONFIGURATION INITIALE")
print("="*60)

# Variables d'environnement
storage_account_name = os.environ.get("AZURE_STORAGE_ACCOUNT_NAME")
storage_account_key = os.environ.get("AZURE_STORAGE_ACCOUNT_KEY")

if storage_account_name and storage_account_key:
    # Configuration Spark
    spark.conf.set(
        f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
        storage_account_key
    )
    print("✅ Configuration Spark effectuée")
else:
    print("⚠️  Variables d'environnement non configurées")

# Nom de la base de données
DATABASE_NAME = "eau_potable"

print(f"📦 Base de données : {DATABASE_NAME}")
print("="*60)

## 2. Vérification des fichiers Parquet dans BRONZE

In [0]:
print("\n" + "="*60)
print("📂 VÉRIFICATION DES FICHIERS PARQUET")
print("="*60)

try:
    bronze_files = dbutils.fs.ls("/mnt/bronze")
    
    # Filtrer les fichiers Parquet consolidés
    plv_files = sorted([f for f in bronze_files if 'DIS_PLV' in f.name and 'consolidated.parquet' in f.name])
    result_files = sorted([f for f in bronze_files if 'DIS_RESULT' in f.name and 'consolidated.parquet' in f.name])
    
    print(f"\n📦 Fichiers DIS_PLV trouvés : {len(plv_files)}")
    for file in plv_files:
        size = file.size / (1024*1024)
        year = file.name.split('_')[2]
        print(f"   ✅ {year} : {file.name} ({size:.2f} MB)")
    
    print(f"\n📦 Fichiers DIS_RESULT trouvés : {len(result_files)}")
    for file in result_files:
        size = file.size / (1024*1024)
        year = file.name.split('_')[2]
        print(f"   ✅ {year} : {file.name} ({size:.2f} MB)")
    
    if not plv_files and not result_files:
        raise Exception("❌ Aucun fichier Parquet consolidé trouvé dans /mnt/bronze")
    
except Exception as e:
    print(f"❌ Erreur : {str(e)}")
    raise

print("\n" + "="*60)

## 3. Création de la base de données

In [0]:
print("\n" + "="*60)
print("🗄️  CRÉATION DE LA BASE DE DONNÉES")
print("="*60)

# Créer la base de données si elle n'existe pas
spark.sql(f"CREATE DATABASE IF NOT EXISTS {DATABASE_NAME}")
print(f"✅ Base de données '{DATABASE_NAME}' créée/vérifiée")

# Définir comme base par défaut
spark.sql(f"USE {DATABASE_NAME}")
print(f"✅ Base de données '{DATABASE_NAME}' activée")

# Afficher les bases existantes
print("\n📋 Bases de données disponibles :")
spark.sql("SHOW DATABASES").show(truncate=False)

print("="*60)

## 4. Création des tables DIS_PLV par année

In [0]:
print("\n" + "="*60)
print("📊 CRÉATION DE LA TABLE DIS_PLV (TOUTES ANNÉES)")
print("="*60)

from pyspark.sql.functions import lit

try:
    all_plv_dfs = []
    plv_stats = []
    
    # Lire tous les fichiers DIS_PLV et ajouter une colonne "annee"
    for file in plv_files:
        year = file.name.split('_')[2]
        
        print(f"\n🔄 Lecture : DIS_PLV {year}")
        print(f"   📂 Fichier : {file.name}")
        
        # Lire le fichier Parquet
        df = spark.read.parquet(file.path)
        
        # Ajouter la colonne "annee"
        df = df.withColumn("annee", lit(year))
        
        row_count = df.count()
        print(f"   📊 Lignes : {row_count:,}")
        
        all_plv_dfs.append(df)
        plv_stats.append({'year': year, 'rows': row_count})
    
    if all_plv_dfs:
        print(f"\n🔗 Fusion de {len(all_plv_dfs)} fichiers DIS_PLV...")
        
        # Fusionner tous les DataFrames
        df_plv_consolidated = all_plv_dfs[0]
        for df in all_plv_dfs[1:]:
            df_plv_consolidated = df_plv_consolidated.union(df)
        
        # Compter le total de lignes
        total_plv_rows = df_plv_consolidated.count()
        print(f"✅ Total de lignes après fusion : {total_plv_rows:,}")
        
        # Créer la table unique
        table_name = "dis_plv"
        print(f"\n💾 Création de la table '{table_name}'...")
        
        df_plv_consolidated.write.mode("overwrite").saveAsTable(f"{DATABASE_NAME}.{table_name}")
        
        print(f"✅ Table '{DATABASE_NAME}.{table_name}' créée avec succès")
        print(f"   📊 Lignes totales : {total_plv_rows:,}")
        print(f"   📊 Colonnes : {len(df_plv_consolidated.columns)}")
        print(f"   📅 Années incluses : {', '.join([s['year'] for s in plv_stats])}")
        
    else:
        print("⚠️  Aucun fichier DIS_PLV à traiter")

except Exception as e:
    print(f"❌ Erreur : {str(e)}")
    raise

print("\n" + "="*60)

## 5. Création des tables DIS_RESULT par année


In [0]:
print("\n" + "="*60)
print("📊 CRÉATION DE LA TABLE DIS_RESULT (TOUTES ANNÉES)")
print("="*60)

try:
    all_result_dfs = []
    result_stats = []
    
    # Lire tous les fichiers DIS_RESULT et ajouter une colonne "annee"
    for file in result_files:
        year = file.name.split('_')[2]
        
        print(f"\n🔄 Lecture : DIS_RESULT {year}")
        print(f"   📂 Fichier : {file.name}")
        
        # Lire le fichier Parquet
        df = spark.read.parquet(file.path)
        
        # Ajouter la colonne "annee"
        df = df.withColumn("annee", lit(year))
        
        row_count = df.count()
        print(f"   📊 Lignes : {row_count:,}")
        
        all_result_dfs.append(df)
        result_stats.append({'year': year, 'rows': row_count})
    
    if all_result_dfs:
        print(f"\n🔗 Fusion de {len(all_result_dfs)} fichiers DIS_RESULT...")
        
        # Fusionner tous les DataFrames
        df_result_consolidated = all_result_dfs[0]
        for df in all_result_dfs[1:]:
            df_result_consolidated = df_result_consolidated.union(df)
        
        # Compter le total de lignes
        total_result_rows = df_result_consolidated.count()
        print(f"✅ Total de lignes après fusion : {total_result_rows:,}")
        
        # Créer la table unique
        table_name = "dis_result"
        print(f"\n💾 Création de la table '{table_name}'...")
        
        df_result_consolidated.write.mode("overwrite").saveAsTable(f"{DATABASE_NAME}.{table_name}")
        
        print(f"✅ Table '{DATABASE_NAME}.{table_name}' créée avec succès")
        print(f"   📊 Lignes totales : {total_result_rows:,}")
        print(f"   📊 Colonnes : {len(df_result_consolidated.columns)}")
        print(f"   📅 Années incluses : {', '.join([s['year'] for s in result_stats])}")
        
    else:
        print("⚠️  Aucun fichier DIS_RESULT à traiter")

except Exception as e:
    print(f"❌ Erreur : {str(e)}")
    raise

print("\n" + "="*60)

In [0]:
# # 🗑️ Suppression des tables annuelles
# print("🗑️  Suppression des tables annuelles...\n")

# tables_to_drop = [
#     'dis_plv_2021', 'dis_plv_2022', 'dis_plv_2023', 'dis_plv_2024', 'dis_plv_2025',
#     'dis_result_2021', 'dis_result_2022', 'dis_result_2023', 'dis_result_2024', 'dis_result_2025'
# ]

# for table in tables_to_drop:
#     try:
#         spark.sql(f"DROP TABLE IF EXISTS eau_potable.{table}")
#         print(f"   ✅ Table '{table}' supprimée")
#     except Exception as e:
#         print(f"   ❌ Erreur: {e}")

# print("\n" + "="*60)
# print("📋 Tables restantes:")
# print("="*60)

# result = spark.sql("SHOW TABLES IN eau_potable").collect()
# print(f"\n   Total: {len(result)} tables\n")
# for row in result:
#     print(f"   ✅ {row.tableName}")

📌 CELLULE 6 : Vérification des tables créées

In [0]:
print("\n" + "="*60)
print("🔍 VÉRIFICATION DES TABLES CRÉÉES")
print("="*60)

# Lister toutes les tables de la base
tables = spark.sql(f"SHOW TABLES IN {DATABASE_NAME}").collect()

print(f"\n📋 Tables dans '{DATABASE_NAME}' :")
print(f"   Total : {len(tables)} tables\n")

for table in tables:
    print(f"   ✅ {table.tableName}")

print("\n" + "="*60)

📌 CELLULE 7 : Statistiques détaillées des tables

In [0]:
print("\n" + "="*60)
print("📊 STATISTIQUES DES TABLES")
print("="*60)

# Statistiques DIS_PLV
print("\n🔵 Table DIS_PLV :")
print("-" * 60)

try:
    df_plv = spark.table(f"{DATABASE_NAME}.dis_plv")
    total_plv = df_plv.count()
    nb_colonnes_plv = len(df_plv.columns)
    
    print(f"   📊 Lignes totales : {total_plv:,}")
    print(f"   📊 Colonnes : {nb_colonnes_plv}")
    
    # Compter par année
    print(f"\n   📅 Répartition par année :")
    df_plv.groupBy("annee").count().orderBy("annee").show()
    
except Exception as e:
    print(f"   ❌ Erreur : {str(e)}")

# Statistiques DIS_RESULT
print("\n🟢 Table DIS_RESULT :")
print("-" * 60)

try:
    df_result = spark.table(f"{DATABASE_NAME}.dis_result")
    total_result = df_result.count()
    nb_colonnes_result = len(df_result.columns)
    
    print(f"   📊 Lignes totales : {total_result:,}")
    print(f"   📊 Colonnes : {nb_colonnes_result}")
    
    # Compter par année
    print(f"\n   📅 Répartition par année :")
    df_result.groupBy("annee").count().orderBy("annee").show()
    
except Exception as e:
    print(f"   ❌ Erreur : {str(e)}")

print("\n" + "="*60)

📌 CELLULE 8 : Schéma des tables

In [0]:
print("\n" + "="*60)
print("📋 SCHÉMA DES TABLES")
print("="*60)

# Schéma DIS_PLV
print(f"\n🔵 Schéma de 'dis_plv' :")
print("-" * 60)

try:
    df_plv = spark.table(f"{DATABASE_NAME}.dis_plv")
    df_plv.printSchema()
    
    print(f"\n📋 Colonnes ({len(df_plv.columns)}) :")
    for idx, col_name in enumerate(df_plv.columns, 1):
        print(f"   {idx:2d}. {col_name}")
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

# Schéma DIS_RESULT
print(f"\n🟢 Schéma de 'dis_result' :")
print("-" * 60)

try:
    df_result = spark.table(f"{DATABASE_NAME}.dis_result")
    df_result.printSchema()
    
    print(f"\n📋 Colonnes ({len(df_result.columns)}) :")
    for idx, col_name in enumerate(df_result.columns, 1):
        print(f"   {idx:2d}. {col_name}")
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

print("\n" + "="*60)

📌 CELLULE 9 : Aperçu des données


In [0]:
print("\n" + "="*60)
print("👀 APERÇU DES DONNÉES")
print("="*60)

# Aperçu DIS_PLV
print(f"\n📄 Aperçu de 'dis_plv' (5 premières lignes) :")
print("-" * 80)

try:
    spark.sql(f"SELECT * FROM {DATABASE_NAME}.dis_plv LIMIT 5").show(truncate=50, vertical=False)
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

# Aperçu DIS_RESULT
print(f"\n📄 Aperçu de 'dis_result' (5 premières lignes) :")
print("-" * 80)

try:
    spark.sql(f"SELECT * FROM {DATABASE_NAME}.dis_result LIMIT 5").show(truncate=50, vertical=False)
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

print("="*60)

📌 CELLULE 10 : Exemples de requêtes SQL


In [0]:
print("\n" + "="*60)
print("💡 EXEMPLES DE REQUÊTES SQL")
print("="*60)

# Exemple 1 : Données PLV pour un prélèvement spécifique
print(f"\n🔵 Exemple 1 : Données PLV pour referenceprel = 00100143925")
print("-" * 80)
try:
    spark.sql(f"""
        SELECT 
            referenceprel,
            cddept,
            nomcommuneprinc,
            dateprel,
            heureprel,
            conclusionprel,
            plvconformitebacterio,
            plvconformitechimique,
            ugelib,
            distrlib
        FROM {DATABASE_NAME}.dis_plv
        WHERE referenceprel = '00100143925'
    """).show(truncate=False, vertical=True)
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

# Exemple 2 : Résultats d'analyses pour ce même prélèvement
print(f"\n🟢 Exemple 2 : Résultats d'analyses pour referenceprel = 00100143925")
print("-" * 80)
try:
    spark.sql(f"""
        SELECT 
            referenceprel,
            cdparametre,
            libmajparametre,
            valtraduite,
            cdunitereference,
            limitequal,
            refqual
        FROM {DATABASE_NAME}.dis_result
        WHERE referenceprel = '00100143925'
        ORDER BY libmajparametre
    """).show(truncate=False)
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

# Exemple 3 : Jointure complète PLV + RESULT pour ce prélèvement
print(f"\n🔗 Exemple 3 : Jointure PLV + RESULT pour referenceprel = 00100143925")
print("-" * 80)
try:
    spark.sql(f"""
        SELECT 
            p.referenceprel,
            p.nomcommuneprinc,
            p.dateprel,
            p.heureprel,
            p.plvconformitebacterio,
            p.plvconformitechimique,
            r.libmajparametre,
            r.valtraduite,
            r.cdunitereference,
            r.limitequal,
            r.refqual
        FROM {DATABASE_NAME}.dis_plv p
        INNER JOIN {DATABASE_NAME}.dis_result r
            ON p.referenceprel = r.referenceprel
            AND p.cddept = r.cddept
        WHERE p.referenceprel = '00100143925'
        ORDER BY r.libmajparametre
    """).show(truncate=False)
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

# Exemple 4 : Nombre de paramètres analysés pour ce prélèvement
print(f"\n📊 Exemple 4 : Statistiques pour referenceprel = 00100143925")
print("-" * 80)
try:
    spark.sql(f"""
        SELECT 
            p.referenceprel,
            p.nomcommuneprinc,
            p.dateprel,
            COUNT(r.cdparametre) as nb_parametres_analyses,
            p.plvconformitebacterio,
            p.plvconformitechimique
        FROM {DATABASE_NAME}.dis_plv p
        LEFT JOIN {DATABASE_NAME}.dis_result r
            ON p.referenceprel = r.referenceprel
            AND p.cddept = r.cddept
        WHERE p.referenceprel = '00100143925'
        GROUP BY 
            p.referenceprel,
            p.nomcommuneprinc,
            p.dateprel,
            p.plvconformitebacterio,
            p.plvconformitechimique
    """).show(truncate=False, vertical=True)
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

# Exemple 5 : Top 10 départements par nombre de prélèvements
print(f"\n🔵 Exemple 5 : Nombre de prélèvements par département (TOP 10)")
print("-" * 80)
try:
    spark.sql(f"""
        SELECT 
            cddept,
            COUNT(*) as nb_prelevements
        FROM {DATABASE_NAME}.dis_plv
        GROUP BY cddept
        ORDER BY nb_prelevements DESC
        LIMIT 10
    """).show()
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

# Exemple 6 : Top 10 paramètres les plus analysés
print(f"\n🟢 Exemple 6 : Top 10 paramètres les plus analysés")
print("-" * 80)
try:
    spark.sql(f"""
        SELECT 
            libmajparametre,
            COUNT(*) as nb_analyses
        FROM {DATABASE_NAME}.dis_result
        GROUP BY libmajparametre
        ORDER BY nb_analyses DESC
        LIMIT 10
    """).show(truncate=50)
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

# Exemple 7 : Jointure générale PLV + RESULT (échantillon)
print(f"\n🔗 Exemple 7 : Jointure PLV + RESULT (échantillon de 10 lignes)")
print("-" * 80)
try:
    spark.sql(f"""
        SELECT 
            p.referenceprel,
            p.nomcommuneprinc,
            p.dateprel,
            r.libmajparametre,
            r.valtraduite,
            r.cdunitereference
        FROM {DATABASE_NAME}.dis_plv p
        INNER JOIN {DATABASE_NAME}.dis_result r
            ON p.referenceprel = r.referenceprel 
            AND p.cddept = r.cddept
        LIMIT 10
    """).show(truncate=40)
except Exception as e:
    print(f"❌ Erreur : {str(e)}")

print("="*60)
print("✅ Exemples de requêtes SQL terminés")
print("="*60)